In [ ]:
# 用于demo一路sEMG&FMG信号进行肌力识别

from motiongui import *

In [ ]:

def gui_start():
    init_window = tkinter.Tk()              #实例化出一个父窗口
    win_a = MotionGuideGUI(init_window)
    # 设置根窗口默认属性
    win_a.set_init_window()
    init_window.after(2000, win_a.gui_loop)
    init_window.mainloop()          #父窗口进入事件循环，可以理解为保持窗口运行，否则界面不展示



In [ ]:
gui_start()